In [1]:
import tensorflow as tf
import numpy as np

In [2]:
@tf.function
def is_phi(element):
    tf.debugging.assert_rank(element, 1)
    
    elem_dim = tf.shape(element)[0]
    phi = tf.one_hot(0, elem_dim)
    
    element = tf.math.l2_normalize(element)
    t = tf.tensordot(element, phi, axes=1)

    return t

test1 = tf.Variable([1,0,0], dtype=tf.float32)
test2 = tf.Variable([0,1,0], dtype=tf.float32)
test3 = tf.Variable([.5,.5,0], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    result1 = is_phi(test1)
    result2 = is_phi(test2)
    result3 = is_phi(test3)

tf.print(result1, tape.gradient(result1, test1))
tf.print(result2, tape.gradient(result2, test2))
tf.print(result3, tape.gradient(result3, test3))

1 [0 0 0]
0 [1 0 0]
0.707106769 [0.707106829 -0.707106709 0]


In [3]:
from library.stacks import stack_push, stack_pop, stack_peek, new_stack, new_stack_from_buffer

In [41]:
# @tf.function
def safe_push(stack, element, is_phi_fn):
    tf.debugging.assert_rank_at_least(stack[0], 2)
    tf.debugging.assert_rank(stack[1], 1)
    tf.debugging.assert_equal(tf.shape(stack[0])[1:], tf.shape(element))
    tf.debugging.assert_equal(tf.rank(stack[0]) - 1, tf.rank(element) )
    
    t = is_phi_fn(element)
    
    old_buffer, old_index = stack
    new_buffer, new_index = stack_push(stack, element)

    buffer = t * old_buffer + (1 - t) * new_buffer
    index = t * old_index + (1 - t) * new_index
    
    tf.print(tokens_pretty_print(old_buffer))
    tf.print(tokens_pretty_print(new_buffer))
    tf.print(tokens_pretty_print(buffer))
    tf.print('-'*80)

    # Hack to tell tensorflow that the shape has not changed
    # TODO: Why does this hack work?
    buffer = tf.reshape(buffer, tf.shape(old_buffer))
    index = tf.reshape(index, tf.shape(old_index))

    new_stack = (buffer, index)

    return new_stack

stack = new_stack((3,3), True)
original_stack = stack

element1 = tf.Variable([0,1,0], dtype=tf.float32)
element2 = tf.Variable([0.5,0.5,0], dtype=tf.float32)
element3 = tf.Variable([0,0,1], dtype=tf.float32)
element4 = tf.Variable([0,1,0], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    stack = safe_push(stack, element1, is_phi)
    stack = safe_push(stack, element2, is_phi)
    stack = safe_push(stack, element3, is_phi)
    stack = safe_push(stack, element4, is_phi)
    
tf.print(stack[0])
tf.print(tf.round(stack[0]))
tf.print(stack[1])
tf.print(tf.round(stack[1]))
tf.print(tape.gradient(stack[0], element3))
tf.print(tape.gradient(stack, original_stack))

_ _ _ 
S _ _ 
S _ _ 
--------------------------------------------------------------------------------
S _ _ 
S _ _ 
S _ _ 
--------------------------------------------------------------------------------
S _ _ 
S O O 
S O O 
--------------------------------------------------------------------------------
S O O 
S O S 
S O S 
--------------------------------------------------------------------------------
[[0 1 0]
 [0.0428932235 0.0428932235 0.707106769]
 [0 0.707106769 0.0857864469]]
[[0 1 0]
 [0 0 1]
 [0 1 0]]
[0.707106769 0.292893231 0]
[1 0 0]
[0.0606601834 0.792893231 0.792893231]
([[0 0 0]
 [0.207106784 0.207106784 0.207106784]
 [0.207106799 0.207106799 0.207106799]], [2.87867975 1.53553391 1.76776707])


In [5]:
@tf.function
def pop_and_purge(stack, phi):
    stack_len = tf.shape(stack[0])[1]
    stack, element = stack_pop(stack)
    stack = stack_push(stack, phi)
    stack, _ = stack_pop(stack)
    
    return stack, element

stack = new_stack_from_buffer(tf.ones((3,3), dtype=tf.float32))
phi = tf.one_hot(0, 3, dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    stack, element = pop_and_purge(stack, phi)
    
tf.print(stack)
tf.print(element)

([[1 1 1]
 [1 1 1]
 [1 0 0]], [0 0 1])
[1 1 1]


In [6]:
stack = new_stack((3,3), True)

element1 = tf.Variable([0,1,0], dtype=tf.float32)
element2 = tf.Variable([1,0,0], dtype=tf.float32)
element3 = tf.Variable([0,0,1], dtype=tf.float32)
element4 = tf.Variable([0,1,0], dtype=tf.float32)

original_stack = stack

with tf.GradientTape(persistent=True) as tape:
    stack = safe_push(stack, element1, is_phi)
    stack = safe_push(stack, element2, is_phi)
    stack = safe_push(stack, element3, is_phi)
    stack = safe_push(stack, element4, is_phi)
    
tf.print(stack[0])
tf.print(stack[1])
tf.print(tape.gradient(stack[0], element3))
tf.print(tape.gradient(stack, original_stack))

[[0 1 0]
 [0 0 1]
 [0 1 0]]
[1 0 0]
[-1 1 1]
([[0 0 0]
 [0 0 0]
 [0 0 0]], [4 1 1])


In [7]:
from library.array_ops import tensor_lookup_2d

In [8]:
TOKEN_DIM = 6
PRODUCTION_DIM = 4
STACK_SIZE = 10
PHI = np.eye(TOKEN_DIM)[0]
S = np.eye(TOKEN_DIM)[1]
O = np.eye(TOKEN_DIM)[2]
T = np.eye(TOKEN_DIM)[3]
X = np.eye(TOKEN_DIM)[4]
PLUS = np.eye(TOKEN_DIM)[5]

E = [PHI, PHI, PHI]

G_s = tf.constant([
    [E, E, E, E, E, E],
    [E, E, E, E, E, E],
    [E, [S, O, T], E, E, E, E],
    [E, [T, PHI, PHI], E, E, E, E],
], dtype=tf.float32)
G_o = tf.constant([
    [E, E, E, [X, PHI, PHI], E, E],
    [E, E, [PLUS, PHI, PHI], E, E, E],
    [E, E, E, E, E, E],
    [E, E, E, E, E, E],
], dtype=tf.float32)
grammar = (G_s, G_o)

In [9]:
def tokens_pretty_print(tokens):
    tokens = tf.argmax(tokens, axis=1)
    lookup = ['_', 'S', 'O', 'T', 'x', '+']
    
    result = ''
    
    for token in tokens:
        result += f'{lookup[token]} '
        
    return result

tokens = tf.transpose(tf.one_hot([0,1,2,3,4,5], TOKEN_DIM, dtype=tf.float32))
tokens_pretty_print(tokens)

'_ S O T x + '

In [30]:
@tf.function
def production_step(grammar, production, stack, output, phi, is_phi_fn):
    tf.debugging.assert_rank(grammar[0], 4)
    tf.debugging.assert_rank(grammar[1], 4)
    tf.debugging.assert_rank(production, 1)
    tf.debugging.assert_rank(stack[0], 2)
    tf.debugging.assert_rank(output[0], 2)
    
    G_s, G_o = grammar
    
    # Save the shapes
    stack_0_shape = tf.shape(stack[0])
    stack_1_shape = tf.shape(stack[1])
    output_0_shape = tf.shape(output[0])
    output_1_shape = tf.shape(output[1])
    
    # Get next token from stack
    stack, stack_top_token = pop_and_purge(stack, phi)

    # Push tokens back onto the stack
    tokens_to_push = tensor_lookup_2d(G_s, production, stack_top_token)
    for token in tf.reverse(tokens_to_push, axis=[0]):
        stack = safe_push(stack, token, is_phi_fn)
    
    # Push tokens to output
    tokens_to_push = tensor_lookup_2d(G_o, production, stack_top_token)
    for token in tokens_to_push:
        output = safe_push(output, token, is_phi_fn)
    
    return stack, output

stack = new_stack(((STACK_SIZE, TOKEN_DIM)))
output = new_stack(((STACK_SIZE, TOKEN_DIM)))

stack = safe_push(stack, tf.constant(S, dtype=tf.float32), is_phi)
production = tf.one_hot(2, PRODUCTION_DIM)
phi = tf.one_hot(0, TOKEN_DIM, dtype=tf.float32)

with tf.GradientTape(persistent = True) as tape:
    tape.watch(grammar)
    tape.watch(production)
    tape.watch(stack)
    tape.watch(output)
    
    new_s, new_o = production_step(grammar, production, stack, output, phi, is_phi)

tf.print(tokens_pretty_print(new_s[0]))
# tf.print(tape.gradient(new_o, output))
# tf.print(tape.gradient(new_s, stack))
# tf.print(tape.gradient(new_s[0], grammar[0]).shape)
# tf.print(tape.gradient(new_s[1], grammar[0]).shape)
# tf.print(tape.gradient(new_o[0], grammar[1]).shape)
# tf.print(tape.gradient(new_o[1], grammar[1]).shape)
tf.print(tape.gradient(new_s, production))

T O S _ _ _ _ _ _ _ 
[-0.0833333358 -0.0833333358 3 -0.0416666679]


In [42]:
stack_shape = (STACK_SIZE, TOKEN_DIM)

phi = tf.one_hot([0], TOKEN_DIM, dtype=tf.float32)
soft_phi = tf.nn.softmax(phi, axis=-1)
stack_buffer = tf.tile(soft_phi, (stack_shape[0], 1))
soft_stack = new_stack_from_buffer(stack_buffer)
output_buffer = tf.tile(soft_phi, (stack_shape[0], 1))
soft_output = new_stack_from_buffer(output_buffer)
soft_s = tf.nn.softmax(tf.constant(S, dtype=tf.float32))

soft_stack = safe_push(soft_stack, soft_s, is_phi)
soft_p = tf.nn.softmax(tf.one_hot(2, PRODUCTION_DIM))

tf.config.experimental_run_functions_eagerly(True)
with tf.GradientTape(persistent = True) as tape:
    tape.watch(grammar)
    tape.watch(production)
    tape.watch(stack)
    tape.watch(output)
    
    # Soften the grammar
    gs, go = grammar
    sgs, sgo = tf.nn.softmax(gs), tf.nn.softmax(go)
    soft_g = (sgs, sgo)

    new_s, new_o = production_step(soft_g, soft_p, soft_stack, soft_output, soft_phi[0], is_phi)
# tf.config.experimental_run_functions_eagerly(False)

tf.print(tokens_pretty_print(soft_stack[0]))
#here
tf.print(tokens_pretty_print(new_s[0]))
# tf.print(tape.gradient(new_o, output))
# tf.print(tape.gradient(new_s, stack))
# tf.print(tape.gradient(new_s[0], grammar[0]).shape)
# tf.print(tape.gradient(new_s[1], grammar[0]).shape)
# tf.print(tape.gradient(new_o[0], grammar[1]).shape)
# tf.print(tape.gradient(new_o[1], grammar[1]).shape)
tf.print(tape.gradient(new_s, production))

_ _ _ _ _ _ _ _ _ _ 
S _ _ _ _ _ _ _ _ _ 
S _ _ _ _ _ _ _ _ _ 
--------------------------------------------------------------------------------
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
--------------------------------------------------------------------------------
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
--------------------------------------------------------------------------------
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
--------------------------------------------------------------------------------
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
--------------------------------------------------------------------------------
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
--------------------------------------------------------------------------------
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ 
-------------------------------------------------------------------------

In [ ]:
def dump_step_info(grammar, production, stack, output):
    gs, go = grammar
    top = stack_peek(stack)
    tf.print('p\t', tf.argmax(production))
    i = tf.argmax(production)
    j = tf.argmax(top)
    tf.print('G_s\t', tokens_pretty_print(gs[i][j]), (i,j))
    tf.print('G_o\t', tokens_pretty_print(go[i][j]), (i,j))
    tf.print('S_i+1\t', tokens_pretty_print(stack[0]), tf.argmax(stack[1]))
    tf.print('O_i+1\t', tokens_pretty_print(output[0]), tf.argmax(output[1]))
    tf.print('-'*80)

In [ ]:
@tf.function
def generate(grammar, productions, stack_shape, S, phi, is_phi_fn, print_steps=False):
    # Reserve space for stack and output
    stack_buffer = tf.tile(phi, (stack_shape[0], 1))
    stack = new_stack_from_buffer(stack_buffer)
    output_buffer = tf.tile(phi, (stack_shape[0], 1))
    output = new_stack_from_buffer(output_buffer)
    
    # Push S to top of stack
    stack = safe_push(stack, S, is_phi)
    
    productions = tf.unstack(productions)

    for production in productions:
        stack, output = production_step(grammar, production, stack, output, phi[0], is_phi_fn)
        
        if print_steps:
            dump_step_info(grammar, production, stack, output)
        
    return tf.reverse(output[0], axis=[0]), stack[0]

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
productions = tf.one_hot([2, 3, 0, 1, 0], PRODUCTION_DIM)

stack_shape = (STACK_SIZE, TOKEN_DIM)
d_S = tf.constant(S, dtype=tf.float32)
d_phi = tf.constant(tf.one_hot([0], TOKEN_DIM))

with tf.GradientTape(persistent = True) as tape:
    tape.watch(productions)
    output, final_stack = generate(grammar, productions, stack_shape, d_S, d_phi, is_phi, True)

tf.config.experimental_run_functions_eagerly(False)
tf.print('Final result:')
tf.print(tokens_pretty_print(output))
tf.print('-'*80)
tf.print('Final stack:')
tf.print(tokens_pretty_print(final_stack))
tf.print('-'*80)
tf.print(tape.gradient(output, productions))

In [ ]:
def encode_to_tokens(s, token_dim, total_length):
    lookup = ['_', 'S', 'O', 'T', 'x', '+']
    arr = []
    for t in s.split(' '):
        arr.append(lookup.index(t))
    
    phi = lookup.index('_')
    arr = ([phi] * (total_length - len(arr))) + arr
        
    return tf.one_hot(arr, token_dim)

encode_to_tokens('x + x +', TOKEN_DIM, 5)

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
productions = tf.one_hot([2, 3, 0, 1, 0], PRODUCTION_DIM)

stack_shape = (STACK_SIZE, TOKEN_DIM)
d_S = tf.constant(S, dtype=tf.float32)
d_phi = tf.constant(tf.one_hot([0], TOKEN_DIM))
expected_output = encode_to_tokens('x + x +', TOKEN_DIM, STACK_SIZE)
zero_stack = tf.one_hot([0] * stack_shape[0], stack_shape[1], dtype=tf.float32)

with tf.GradientTape(persistent = True) as tape:
    tape.watch(productions)
    # Soften the grammar
    gs, go = grammar
    sgs, sgo = tf.nn.softmax(gs), tf.nn.softmax(go)
    soft_g = (sgs, sgo)

    # Soften the productions
    soft_p = tf.nn.softmax(productions,axis=-1)

    # Soften S
    soft_s = tf.nn.softmax(d_S)

    soft_phi = tf.nn.softmax(d_phi, axis=-1)
    
    output_, stack_ = generate(soft_g, soft_p, stack_shape, soft_s, soft_phi, is_phi, True)

    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(output, output_))
    loss += tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(zero_stack, stack_))

tf.config.experimental_run_functions_eagerly(False)
tf.print('Final result:')
tf.print(tokens_pretty_print(output_))
tf.print('-'*80)
tf.print('Final stack:')
tf.print(tokens_pretty_print(stack_))
tf.print('-'*80)
tf.print(loss)
tf.print(tape.gradient(loss, productions))

In [ ]:
opt = tf.keras.optimizers.Adam(1e-1)

@tf.function
def train_step(grammar, productions, stack_shape, S, is_phi_fn, output, print_steps=False):
    zero_stack = tf.one_hot([0] * stack_shape[0], stack_shape[1], dtype=tf.float32)
    phi = tf.one_hot([0], stack_shape[1], dtype=tf.float32)
    
    with tf.GradientTape() as tape:
        tape.watch(productions)
#         # Soften the grammar
#         gs, go = grammar
#         sgs, sgo = tf.nn.softmax(gs), tf.nn.softmax(go)
#         soft_g = (sgs, sgo)
        
#         # Soften the productions
#         soft_p = tf.nn.softmax(productions,axis=-1)
        
#         # Soften S
#         soft_s = tf.nn.softmax(S)
        
#         soft_phi = tf.nn.softmax(phi, axis=-1)
        
#         output_, stack_ = generate(soft_g, soft_p, stack_shape, soft_s, soft_phi, is_phi_fn, True)
        
#         loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(output, output_))
#         loss += tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(zero_stack, stack_))
        
        output_, stack_ = generate(grammar, productions, stack_shape, S, phi, is_phi_fn, print_steps)
        loss = tf.nn.l2_loss(output - output_) + tf.nn.l2_loss(zero_stack - stack_)
        
    grads = tape.gradient(loss, productions)
    opt.apply_gradients(zip([grads], [productions]))
    
    return loss, output_, stack_

MAX_PRODUCTIONS = 5
# productions = tf.Variable(tf.one_hot([0] * MAX_PRODUCTIONS, PRODUCTION_DIM), dtype=tf.float32)
productions = tf.Variable(tf.one_hot([2, 3, 0, 1, 0], PRODUCTION_DIM, dtype=tf.float32))
# productions = tf.Variable(tf.one_hot([2, 0, 0, 0, 0], PRODUCTION_DIM), dtype=tf.float32)
stack_shape = (STACK_SIZE, TOKEN_DIM)
d_S = tf.constant(S, dtype=tf.float32)
output = encode_to_tokens('x + x', TOKEN_DIM, STACK_SIZE)

tf.config.experimental_run_functions_eagerly(True)
loss, output_, stack_ = train_step(grammar, productions, stack_shape, d_S, is_phi, output, True)
tf.config.experimental_run_functions_eagerly(False)
tf.print(loss)
tf.print(tokens_pretty_print(output_), tokens_pretty_print(output))
tf.print(tokens_pretty_print(stack_))

In [ ]:
# productions = tf.Variable(tf.one_hot([2, 3, 0, 1, 0], PRODUCTION_DIM), dtype=tf.float32)
productions = tf.Variable(tf.one_hot([2, 3, 0, 0, 0], PRODUCTION_DIM), dtype=tf.float32)
output = encode_to_tokens('x + x', TOKEN_DIM, STACK_SIZE)

for var in opt.variables():
    var.assign(tf.zeros_like(var))

for i in range(100):
    loss, output_, stack_ = train_step(grammar, productions, stack_shape, d_S, is_phi, output)
    if i % 10 == 0:
        p_output = tokens_pretty_print(output_)
        p_stack = tokens_pretty_print(stack_)
        
        tf.print(loss, p_output, p_stack, tf.argmax(productions, axis=-1))
